<a href="https://colab.research.google.com/github/andreacangiani/NSPDE-ANA22/blob/main/python/C7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FEniCSx Tutorial 2: solving a diffusion problem in 2D

Load FEniCSx modules

In [ ]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/20faf6e/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
try:
    import pyvista
except ImportError:
    !pip3 install itkwidgets==0.32.1 pyvista==0.33.2
    import pyvista
finally:
    import google.colab
    google.colab.output.enable_custom_widget_manager()

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@bdc5d58"
    import multiphenicsx

In [ ]:
import dolfinx.fem
import dolfinx.mesh
import mpi4py
import multiphenicsx.io
import numpy as np
import ufl

We consider the model boundary value problem: find $u: \Omega = (0, 1)^2 \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \Delta u = f, & \text{in } \Omega,\\
u = g, & \text{on } \partial\Omega.
\end{cases}
\end{equation*}

with $f\equiv 1$ and the boundary value $g(\mathbf{x})$ given by
$$
g(\mathbf{x}) = g(x_0, x_1) = \sin(3 \pi x_0 + 1) \ \sin(3 \pi x_1 + 1).
$$


**Task 1: create a mesh.** As first example, we generate a triangular mesh of the domain $\Omega$, dividing both the horizontal and vertical sides of the square in nxm equispaced subintervals.

Similarly to 1D case, `dolfinx.mesh` provides the function `create_unit_square` for this task. 

In [ ]:
n =
m =
mesh = 

Let's see how many cells are in the mesh. 

We store in:
* tdim: the problem dimension (2)
* fdim: the mesh scheleton entities dimension (1)
* num_cells: the number of triangles in the mesh

In [ ]:
tdim = 
fdim = 

num_cells

We can obtain an interactive plot of the domain using `pyvista`. (Click on the menu: next to the dropdown that contains "Geometry 0" you may find three different representations: the domain itself, the edges of the mesh, and both overlayed.)

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

**Task 2:** Determine IDs of boundary edges in view of the application of the Dirichlet boundary condition.

As in 1D case, this is obtained via the `dolfinx.mesh` `locate_entities_boundary` function. We want all edges on the boundary, but the function always requires a third input to permit the selection of parts of the boundary. As a workaround to this, we pass as third argument the function which always returns `True`.

In [ ]:
boundary_entities = 


and we can visualise the found boundary entities to check this was done correctly.

In [ ]:
multiphenicsx.io.plot_mesh_entities(mesh, fdim, boundary_entities)

**Task 3: create FEM space.**

Define the finite element function space $V_h$ using $\mathbb{P}_1$ Lagrange elements.

This part of the code is indistinguishable from the 1D version...

In [ ]:
Vh = 

... and compute its dimension

In [ ]:
Vh_dim = 

Once the FE space is at hand, we introduce ufl symbols to define the trial and test functions for our weak formulation:

In [ ]:
uh = 
vh = 

**Task 4:** set up FEM system

We produce the weak formulation of the problem: find $u_h\in V_h$ such that
$$ \int_\Omega \nabla u \cdot \nabla v \ \mathrm{d} \mathbf{x} = \int_\Omega f \ v \ \mathrm{d} \mathbf{x},\qquad\forall v_h\in V_h$$
using `ufl`.

**Task 5:** set up the boundary conditions

In order to assign the boundary condition we first need to evaluate the expression of $g$ (i.e. $\sin(3 \pi x_0 + 1) \ \sin(3 \pi x_1 + 1)$ in our case) on the finite element space $V_h$. We do this by interpolation, so we define the discrete boundary condition as 
$$g_h=I_h g$$ 

with $I_h$ the interpolation operator.

In [ ]:
gh = 

We then initialize a `dolfinx.fem` `dirichletbc` object, stating that the Dirichlet boundary condition should be equal to `gh` on each facet in `boundary_entities`.

In [ ]:
boundary_dofs = 
bc = 

**Task 6:** solve the FEM system.

As in 1D case, we have to first provide a `Function` class to store the solution of a finite element problem and then we are ready to solve the discrete problem allocating a new `LinearProblem` (which uses `PETSc`), providing as input the bilinear form `a`, the linear functional `F`, the boundary conditions `bcs`, and where to store the solution. Further solver options can also be passed to `PETSc`.

In [ ]:
solution = 

In [ ]:
problem = dolfinx.fem.petsc.LinearProblem(
    a, F, bcs=[bc], u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

**Task7:** plot the solution

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h")

And here is a3D plot:

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h", warp_factor=0.1)

# Working with subdomains

We now solve the same problem but with the forcing function given by:

$$
f(\mathbf{x}) = \begin{cases}
50, & \mathbf{x} \in [0.2, 0.8]^2,\\
1, & \text{otherwise},
\end{cases}
$$


We shall code this using by defining two subdomains:
$$\Omega_0 = [0.2, 0.8]^2 \quad \text{and} \quad \Omega_1 = \Omega \setminus \Omega_0$$ 

 In order to do so, we need to ensure that the mesh is alligned with the subdomains. For instance, we can take $n=m=10$: 

In [ ]:
n = 10
m = 10
mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

num_cells = mesh.topology.index_map(tdim).size_local
num_cells

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

**New Task:** Define the two subdomains $\Omega_0$ and $\Omega_1$ in view of the implementation of the forcing function $f$.

*   mark $\Omega_0$ and $\Omega_1$ with labels `0` and `1`
*   use the `dolfinx.mesh` function `locate_entities` to determine whether a cell is in $\Omega_0$. This function checks each of the three vertices of the triangular cell, and locates all cells in which the provided condition is satisfied on all three vertices.

We label each cell in $\Omega_0$ with the label `0` by using the `np.full` function which return a new array with shape of input filled with value.

In [ ]:
cells_0_labels = 

The remaining cells will belong to $\Omega_1$.

To find them we use `np.arange(ar1,ar2)` which finds gives the set difference of two arrays (return the unique values in `ar1` that are not in `ar2`).

In [ ]:
cells_1 =

In [ ]:
cells_1_labels = 

We then store both subdomains in a `dolfinx.mesh` `MeshTags` object.

In [ ]:
subdomains = dolfinx.mesh.MeshTags(
    mesh, tdim,
    np.hstack((cells_1, cells_0)),
    np.hstack((cells_1_labels, cells_0_labels)))

We finally plot with `pyvista` the subdomains to verify the correct assignment of the label.

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

Defining the FEM space is as before:

In [ ]:
Vh = 

In [ ]:
uh = 
vh = 

In order to write this in `ufl`, we need to inform the integral measure `dx` of the subdomain labels, using `ufl.Measure` and providing as data the `subdomains` object that we have created.

In [ ]:
dx =

The bilinear form is straighforward:

In [ ]:
a =

In [ ]:
F = 

The rest is as before:

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h", warp_factor=0.1)

**Exercise 1:** Use the subdomain idea to solve the problem: find $u: \Omega = (0, 1)^2 \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \nabla (\kappa \nabla u) = f, & \text{in } \Omega,\\
u = g, & \text{on } \partial\Omega.
\end{cases}
\end{equation*}
with $f$  and $g$ as before and
$$
\kappa(\mathbf{x}) = \begin{cases}
1, & \mathbf{x} \in [0.2, 0.8]^2,\\
0.1, & \text{otherwise},
\end{cases}
$$


**Exercise 1:** By yourself or using the official tutorial

https://jorgensd.github.io/dolfinx-tutorial/index.html

as reference, find out how to use the subdomain idea to define different boundary conditions. For instance, solve with:
$$
u = g \quad\text{if } \quad x=0 \quad \text{or}\quad  x=1,
$$
and 
$$
-\frac{\partial u}{\partial {\mathbf n}} = h \quad \text{otherwise}.
$$
with
$$
h(x,y)=
\left\{
\begin{array}{ll}
0 & \text{if } y=0\\
-4 & \text{if } y=1.
\end{array}
\right.
$$

